# API Research & Testing Notebook

Use this invalid notebook to research and test new tools for the API.
It is located in the `deploy` folder and can import modules from it directly.

In [2]:
import sys
import os

# Add current directory to path so we can import local modules
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)

print(f"Current working directory: {current_dir}")

# Import modules from deploy package
import config
import login_test
import main_fastapi

print("Modules imported successfully!")

In [ ]:
# Example: Test getting profile
try:
    profile = login_test.get_kite_profile()
    if profile:
        print("Profile fetched successfully:", profile['user_name'])
    else:
        print("Could not fetch profile. Try hitting the login endpoint first.")
except Exception as e:
    print(f"Error: {e}")

In [6]:
import load_kite_from_access

In [7]:
# --- Research Scratchpad ---
# Write your test code here
kite = load_kite_from_access.kite
print(kite)

In [ ]:
def get_orders(kite):
    """Fetch and print orders."""
    try:
        orders = kite.orders()
        print(f"Fetched {len(orders)} orders.")
        return orders
    except Exception as e:
        print(f"Error fetching orders: {e}")
        return None

def get_holdings(kite):
    """Fetch and print holdings."""
    try:
        holdings = kite.holdings()
        print(f"Fetched {len(holdings)} holdings.")
        return holdings
    except Exception as e:
        print(f"Error fetching holdings: {e}")
        return None

def get_positions(kite):
    """Fetch and print positions."""
    try:
        positions = kite.positions()
        print(f"Fetched positions: {positions.keys()}")
        return positions
    except Exception as e:
        print(f"Error fetching positions: {e}")
        return None

def place_stop_loss_gtt(kite, tradingsymbol, exchange, trigger_percent, quantity=1, product=None):
    """
    Places a GTT Stop Loss order.
    
    Args:
        kite: KiteConnect instance
        tradingsymbol: Trading symbol (e.g., 'INFY', 'CRUDEOIL23OCTFUT')
        exchange: Exchange (e.g., 'NSE', 'MCX')
        trigger_percent: Percentage to drop (e.g., 5 for 5% SL). ENTER POSITIVE NUMBER.
        quantity: Quantity to sell
        product: Order product (default is CNC/NRML based on segment, but best to specify)
    """
    try:
        # 1. Fetch LTP to calculate trigger price
        instrument_key = f"{exchange}:{tradingsymbol}"
        ltp_data = kite.ltp(instrument_key)
        
        if not ltp_data or instrument_key not in ltp_data:
            print(f"❌ Error: Could not fetch LTP for {instrument_key}")
            return None
            
        last_price = ltp_data[instrument_key]['last_price']
        print(f"Current LTP for {tradingsymbol}: {last_price}")
        
        # 2. Calculate Trigger Price
        # For Sell SL, trigger is BELOW LTP
        # Example: LTP 100, 5% SL -> Trigger at 95
        trigger_price = last_price * (1 - (trigger_percent / 100))
        trigger_price = round(trigger_price * 20) / 20  # Round to nearest tick (0.05)
        
        print(f"Calculated SL Trigger Price ({trigger_percent}% drop): {trigger_price}")
        
        # 3. Place GTT
        trigger_type = kite.GTT_TYPE_SINGLE
        
        if not product:
             product = kite.PRODUCT_CNC if exchange == 'NSE' else kite.PRODUCT_NRML

        # Order to be placed when triggered
        orders = [
            {
                "exchange": exchange,
                "tradingsymbol": tradingsymbol,
                "transaction_type": kite.TRANSACTION_TYPE_SELL,
                "quantity": quantity,
                "order_type": kite.ORDER_TYPE_LIMIT,
                "product": product,
                "price": trigger_price, 
            }
        ]
        
        # Place GTT
        gtt_id = kite.place_gtt(
            trigger_type=trigger_type,
            tradingsymbol=tradingsymbol,
            exchange=exchange,
            trigger_values=[trigger_price],
            last_price=last_price,
            orders=orders
        )
        
        print(f"✅ GTT Order Placed. ID: {gtt_id}")
        return gtt_id
        
    except Exception as e:
        print(f"❌ Error placing GTT: {e}")
        return None

In [ ]:
import json
import os
import time

class PriceMonitor:
    """
    Background service to fetch prices for a list of tracked symbols.
    Stores latest prices in a dictionary.
    """
    def __init__(self, kite):
        self.kite = kite
        self.watchlist = set() # Set of 'exchange:symbol' strings
        self.latest_prices = {} # Dict: {'exchange:symbol': price}

    def add_symbol(self, exchange, symbol):
        """Add a symbol to the watchlist."""
        key = f"{exchange}:{symbol}"
        self.watchlist.add(key)
        print(f"Start tracking: {key}")

    def remove_symbol(self, exchange, symbol):
        """Remove a symbol from the watchlist."""
        key = f"{exchange}:{symbol}"
        if key in self.watchlist:
            self.watchlist.remove(key)
            if key in self.latest_prices:
                del self.latest_prices[key]
            print(f"Stop tracking: {key}")

    def update_prices(self):
        """Fetch fresh prices for all symbols in watchlist."""
        if not self.watchlist:
            return
            
        try:
            # Bulk fetch
            # print(f"Fetching prices for {len(self.watchlist)} symbols...")
            ltp_data = self.kite.ltp(list(self.watchlist))
            
            for key, data in ltp_data.items():
                self.latest_prices[key] = data['last_price']
                
            # print(f"Prices updated at {time.strftime('%H:%M:%S')}")
        except Exception as e:
            print(f"❌ Error updating prices: {e}")

    def get_price(self, exchange, symbol):
        """Get the latest fetched price safely."""
        key = f"{exchange}:{symbol}"
        return self.latest_prices.get(key)


class TrailingStopLossManager:
    """
    Manages Trailing Stop Loss using GTT orders and a local JSON disk cache.
    Tracks the Highest Point (High Water Mark - HWM) of the LTP.
    Consumer of PriceMonitor data.
    """
    def __init__(self, kite, cache_file="tsl_cache.json"):
        self.kite = kite
        self.cache_file = cache_file
        self.cache = self.load_cache()

    def load_cache(self):
        """Load HWM data from disk."""
        if os.path.exists(self.cache_file):
            try:
                with open(self.cache_file, 'r') as f:
                    return json.load(f)
            except Exception as e:
                print(f"Error loading cache: {e}")
                return {}
        return {}

    def save_cache(self):
        """Save HWM data to disk."""
        try:
            with open(self.cache_file, 'w') as f:
                json.dump(self.cache, f, indent=4)
        except Exception as e:
            print(f"Error saving cache: {e}")

    def update_trailing_stop_loss(self, tradingsymbol, exchange, trailing_percent, quantity=1, product=None, gtt_id=None, current_ltp=None):
        """
        Updates the Trailing Stop Loss for a given symbol.
        Requires `current_ltp` to be passed (e.g. from PriceMonitor).
        """
        instrument_key = f"{exchange}:{tradingsymbol}"
        
        # Validate incoming price
        if current_ltp is None:
            # Fallback: fetch manually if needed, but warn
            print(f"⚠️ Warning: No price provided for {tradingsymbol}, fetching manually.")
            ltp_data = self.kite.ltp(instrument_key)
            if instrument_key in ltp_data:
                current_ltp = ltp_data[instrument_key]['last_price']
            else:
                print(f"❌ Failed to fetch price for {tradingsymbol}")
                return

        # 2. Get or Initialize Cache Entry
        if instrument_key not in self.cache:
            print(f"Creating new TSL entry for {tradingsymbol} at LTP {current_ltp}")
            self.cache[instrument_key] = {
                "hwm_price": current_ltp,
                "gtt_id": gtt_id, # Can be None initially if not provided
                "last_trigger_price": 0,
                "symbol": tradingsymbol,
                "exchange": exchange
            }
        
        entry = self.cache[instrument_key]
        
        # Update HWM if current price is higher
        if current_ltp > entry["hwm_price"]:
            # print(f"🚀 New High for {tradingsymbol}! Old HWM: {entry['hwm_price']} -> New HWM: {current_ltp}")
            entry["hwm_price"] = current_ltp
            self.save_cache() # Save immediately
            
        hwm_price = entry["hwm_price"]
        
        # 3. Calculate New SL Price
        new_sl_price = hwm_price * (1 - (trailing_percent / 100))
        new_sl_price = round(new_sl_price * 20) / 20 # Round to tick
        
        current_trigger_price = entry.get("last_trigger_price", 0)
        
        # 4. Check if we need to modify GTT
        if new_sl_price > current_trigger_price:
            print(f"⚡ Trailing Up {tradingsymbol}! HWM: {hwm_price}, New SL: {new_sl_price} (Old: {current_trigger_price})")
            
            active_gtt_id = entry.get("gtt_id") or gtt_id
            
            if not active_gtt_id:
                # Place NEW GTT
                print("Placing Initial GTT...")
                new_gtt_id = place_stop_loss_gtt(
                    self.kite, tradingsymbol, exchange, trailing_percent, quantity=quantity, product=product
                )
                if new_gtt_id:
                    entry["gtt_id"] = new_gtt_id
                    entry["last_trigger_price"] = new_sl_price
                    self.save_cache()
            else:
                # MODIFY Existing GTT
                try:
                    self.kite.modify_gtt(
                        trigger_id=active_gtt_id,
                        trigger_type=self.kite.GTT_TYPE_SINGLE,
                        tradingsymbol=tradingsymbol,
                        exchange=exchange,
                        trigger_values=[new_sl_price],
                        last_price=current_ltp,
                        orders=[{
                            "exchange": exchange,
                            "tradingsymbol": tradingsymbol,
                            "transaction_type": self.kite.TRANSACTION_TYPE_SELL,
                            "quantity": quantity,
                            "order_type": self.kite.ORDER_TYPE_LIMIT,
                            "product": product if product else self.kite.PRODUCT_CNC,
                            "price": new_sl_price
                        }]
                    )
                    print(f"✅ Modified GTT {active_gtt_id} to Trigger: {new_sl_price}")
                    entry["last_trigger_price"] = new_sl_price
                    self.save_cache()
                except Exception as e:
                    print(f"❌ Error modifying GTT {active_gtt_id}: {e}")
                    if "InputException" in str(e) or "not found" in str(e).lower():
                        print("GTT ID invalid, clearing from cache.")
                        entry["gtt_id"] = None
                        self.save_cache()


In [8]:
# Test the functions
if kite:
    print("\n--- Orders ---")
    orders = get_orders(kite)
    if orders:
        print(orders[:1]) # Print first order as sample
    
    print("\n--- Holdings ---")
    holdings = get_holdings(kite)
    if holdings:
        print(holdings[:1]) # Print first holding as sample
    
    print("\n--- Positions ---")
    positions = get_positions(kite)
    if positions:
        print("Net positions count:", len(positions.get('net', [])))
        print("Day positions count:", len(positions.get('day', [])))
        
    # --- Test TSL Refactored Logic ---
    
    # 1. Setup Monitor and TSL Manager
    monitor = PriceMonitor(kite)
    tsl_manager = TrailingStopLossManager(kite)

    # 2. Define Positions to Track
    # You can loop here indefinitely or schedule this function
    my_positions = [
         {'symbol': 'INFY', 'exchange': 'NSE', 'percent': 5, 'quantity': 10},
         {'symbol': 'TCS', 'exchange': 'NSE', 'percent': 4, 'quantity': 5}
    ]
    
    # Add symbols to monitor
    for p in my_positions:
        monitor.add_symbol(p['exchange'], p['symbol'])
    
    # 3. Simulate Loop (Run this manually or in a loop)
    print("\n--- Monitoring Loop Step ---")
    
    # A. Update Prices (The 'Background' Job)
    monitor.update_prices()
    
    # B. Process TSL Logic using updated prices
    for p in my_positions:
        price = monitor.get_price(p['exchange'], p['symbol'])
        if price:
            tsl_manager.update_trailing_stop_loss(
                tradingsymbol=p['symbol'],
                exchange=p['exchange'],
                trailing_percent=p['percent'],
                quantity=p['quantity'],
                current_ltp=price  # Pass the fetched price!
            )
    
else:
    print("Kite object is not available.")